In [1]:
import gempyor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import glob, os, sys, re, shutil
from pathlib import Path

# import seaborn as sns
import matplotlib._color_data as mcd
import pyarrow.parquet as pq
import click
import subprocess
import dask.dataframe as dd
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
from matplotlib.backends.backend_pdf import PdfPages

import os
import multiprocessing as mp
import pandas as pd
import pyarrow.parquet as pq
from gempyor import config, model_info, outcomes, seir


data_dir = "RSV_USA/"
import os
os.chdir(data_dir)
data_dir = "."  # necessary for now for time serie parameters !!! fixme 

In [2]:
config_path = f"config_rsv_simplified_inference_v3_MD.yml"
run_id = "config_rsv_simplified_inference_v3_MD.yml"

config.clear()
config.read(user=False)
config.set_file(config_path)
print(config_path)


seir_modifiers_scenario="none"
outcome_modifiers_scenario="IHRadj"

in_run_id = run_id
out_run_id = in_run_id
in_prefix = "emcee"

out_prefix = in_prefix

write_csv = False
write_parquet = True
modinf = model_info.ModelInfo(
    config=config,
    nslots=1,
    seir_modifiers_scenario=seir_modifiers_scenario,
    outcome_modifiers_scenario=outcome_modifiers_scenario,
    write_csv=write_csv,
    write_parquet=write_parquet,
    first_sim_index=1,
    in_run_id=in_run_id,
    in_prefix=in_prefix,
    inference_filename_prefix="no",
    inference_filepath_suffix="no",
    out_run_id=out_run_id,
    out_prefix=out_prefix,
    stoch_traj_flag=False,
)

CRITICAL:root:No mobility matrix specified -- assuming no one moves


config_rsv_simplified_inference_v3_MD.yml


In [3]:
# find what to perturb


fitted_params = {
    "ptype":[],
    "pname":[],
    "pdist":[],
    "ub":[],
    "lb":[],
}
n_params=0

print("SEIR MODIFIERS")
for npi in gempyor.config["seir_modifiers"]["modifiers"].get():
    if gempyor.config["seir_modifiers"]["modifiers"][npi]["perturbation"].exists():
        c = config["seir_modifiers"]["modifiers"][npi]
        fitted_params["ptype"].append("snpi")
        fitted_params["pname"].append(npi)
        fitted_params["pdist"].append(c["value"].as_random_distribution())
        fitted_params["lb"].append(c["value"]["a"].get())
        fitted_params["ub"].append(c["value"]["b"].get())
        print(f" >> {npi} has perturbation, recording")
        n_params+=len(modinf.subpop_struct.subpop_names)
        
print("OUTCOMES MODIFIERS")   
for npi in gempyor.config["outcome_modifiers"]["modifiers"].get():
    if gempyor.config["outcome_modifiers"]["modifiers"][npi]["perturbation"].exists():
        c = config["outcome_modifiers"]["modifiers"][npi]
        fitted_params["ptype"].append("hnpi")
        fitted_params["pname"].append(npi)
        fitted_params["pdist"].append(c["value"].as_random_distribution())
        fitted_params["lb"].append(c["value"]["a"].get())
        fitted_params["ub"].append(c["value"]["b"].get())
        print(f" >> {npi} has perturbation, recording")
        n_params+=len(modinf.subpop_struct.subpop_names)

SEIR MODIFIERS
 >> Ro_mod has perturbation, recording
 >> covid_mod has perturbation, recording
 >> omicron_mod has perturbation, recording
 >> seas_amp_mod has perturbation, recording
 >> seas_offset_mod has perturbation, recording
OUTCOMES MODIFIERS
 >> IHRadj has perturbation, recording


In [9]:
# Find the ground-truth
gt = pd.read_csv(f"{data_dir}/"+gempyor.config["inference"]["gt_data_path"].get())
# gt
statistics = {}
# # Ingoring agreegation and all, assuming by weekP
for stat in gempyor.config["inference"]["statistics"]:
    statistics[gempyor.config["inference"]["statistics"][stat]["sim_var"].get()] = gempyor.config["inference"]["statistics"][stat]["data_var"].get()
statistics
gt = gempyor.read_df(gempyor.config["inference"]["gt_data_path"].get())
gt["date"]= pd.to_datetime(gt['date'])
gt = gt.set_index("date")

## Create the first gempyor object

In [4]:

print("MAKING A TEST RUN TO GET SETUP")
(
    unique_strings,
    transition_array,
    proportion_array,
    proportion_info,
) = modinf.compartments.get_transition_array()

outcomes_parameters = outcomes.read_parameters_from_config(modinf)


npi_seir = seir.build_npi_SEIR(
    modinf=modinf, load_ID=False, sim_id2load=None, config=config
)
if modinf.npi_config_outcomes:
    npi_outcomes = outcomes.build_outcome_modifiers(
                modinf=modinf,
                load_ID=False,
                sim_id2load=None,
                config=config,
            )

p_draw = modinf.parameters.parameters_quick_draw(
                n_days=modinf.n_days, nsubpops=modinf.nsubpops
            )

initial_conditions = modinf.initial_conditions.draw(sim_id=0, setup=modinf)
seeding_data, seeding_amounts = modinf.seeding.draw(sim_id=0, setup=modinf)



# reduce them
parameters = modinf.parameters.parameters_reduce(p_draw, npi_seir)
        # Parse them
parsed_parameters = modinf.compartments.parse_parameters(
    parameters, modinf.parameters.pnames, unique_strings
)

states = seir.steps_SEIR(
    modinf,
    parsed_parameters,
    transition_array,
    proportion_array,
    proportion_info,
    initial_conditions,
    seeding_data,
    seeding_amounts,
)

seir_out_df_ref = seir.postprocess_and_write(
    0, modinf, states, p_draw, npi_seir, seeding_data
)
snpi_df_ref = npi_seir.getReductionDF()

outcomes_df, hpar_df = outcomes.compute_all_multioutcomes(
    modinf=modinf,
    sim_id2write=0,
    parameters=outcomes_parameters,
    loaded_values=None,
    npi=npi_outcomes,
    bypass_seir=seir_out_df_ref
)
outcomes_df_ref, hpar_ref, hnpi_df_ref = outcomes.postprocess_and_write(
    sim_id=0,
    modinf=modinf,
    outcomes_df=outcomes_df,
    hpar=hpar_df,
    npi=npi_outcomes,
)
print("TEST RUN IS DONE")

INFO:root:[SEIR.compartments] completed in 1.16 s
INFO:root:[Outcome.structure] completed in 0.00 s
INFO:root:[SEIR.NPI] completed in 0.12 s
INFO:root:[Outcomes.Modifiers] completed in 0.02 s
/Users/chadi/Research/FlepiMoP_Projects/flepiMoP/flepimop/gempyor_pkg/src/gempyor/seeding_ic.py:492: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  y0[comp_idx, pl_idx] = float(ic_df_compartment_val)
INFO:root:Integrating with method rk4.jit
13819it [00:23, 579.47it/s]


In [7]:
def run_simulation(snpi_df_in, hnpi_df_in):
    npi_seir = seir.build_npi_SEIR(
        modinf=modinf, load_ID=False, sim_id2load=None, config=config, 
        bypass_DF=snpi_df_in
    )
    if modinf.npi_config_outcomes:
        npi_outcomes = outcomes.build_outcome_modifiers(
                    modinf=modinf,
                    load_ID=False,
                    sim_id2load=None,
                    config=config,
                    bypass_DF=hnpi_df_in
                )
        

    # reduce them
    parameters = modinf.parameters.parameters_reduce(p_draw, npi_seir)
            # Parse them
    parsed_parameters = modinf.compartments.parse_parameters(
        parameters, modinf.parameters.pnames, unique_strings
    )

    states = seir.steps_SEIR(
        modinf,
        parsed_parameters,
        transition_array,
        proportion_array,
        proportion_info,
        initial_conditions,
        seeding_data,
        seeding_amounts,
    )

    seir_out_df = seir.postprocess_and_write(
        0, modinf, states, p_draw, npi_seir, seeding_data
    )

    outcomes_df, hpar_df = outcomes.compute_all_multioutcomes(
        modinf=modinf,
        sim_id2write=0,
        parameters=outcomes_parameters,
        loaded_values=None,
        npi=npi_outcomes,
        bypass_seir=seir_out_df
    )
    outcomes_df, hpar, hnpi = outcomes.postprocess_and_write(
        sim_id=0,
        modinf=modinf,
        outcomes_df=outcomes_df,
        hpar=hpar_df,
        npi=npi_outcomes,
    )
    outcomes_df = outcomes_df.set_index("date")

    return outcomes_df


def compute_likelyhood(model_df, gt):
    log_loss = 0
    for subpop in modinf.subpop_struct.subpop_names:
        gt_s = gt[gt["subpop"]==subpop].loc[modinf.ti:modinf.tf].drop(["subpop"],axis=1).resample("W-SAT").sum()
        model_df_s = model_df[model_df["subpop"]==subpop].drop(["subpop","time"],axis=1).loc[min(gt_s.index):max(gt_s.index)].resample("W-SAT").sum() # todo sub subpop here
        for key, value in statistics.items():
            assert model_df_s[key].shape == gt_s[value].shape
            log_loss += sum(np.log((model_df_s[key]-gt_s[value])**2))

    return log_loss


def check_in_bounds(proposal):
    proposal = np.random.random(len(fitted_params["ptype"]))*1.5
    if (proposal < fitted_params["lb"]).any() or (proposal > fitted_params["ub"]).any():
        return False
    return True


SyntaxError: unmatched ')' (170123325.py, line 66)

In [ ]:
if False:
    outcomes_df = run_simulation(snpi_df_in=snpi_df_ref, hnpi_df_in=hnpi_df_ref)
    outcomes_df_ref.set_index("date")["incidH"].plot(lw=2, c='k')
    outcomes_df.set_index("date")["incidH"].plot(lw=1, c='r')

In [ ]:
def get_log_prob(proposal):
    if not check_in_bounds(proposal=proposal):
        return np.inf
    
    snpi_df_mod = snpi_df_ref.copy(deep=True)
    hnpi_df_mod = hnpi_df_ref.copy(deep=True)

    for i in range(n_params):
        if fitted_params["ptype"][i] == "snpi":
            snpi_df_mod.loc[snpi_df_mod["npi_name"] == fitted_params["pname"][i],"reduction"] = proposal[i]
        elif fitted_params["ptype"][i] == "hnpi":
            hnpi_df_mod.loc[hnpi_df_mod["npi_name"] == fitted_params["pname"][i],"reduction"] = proposal[i]

    outcomes_df = run_simulation(snpi_df_in=snpi_df_mod, hnpi_df_in=hnpi_df_mod)

    llik = compute_likelyhood(model_df=outcomes_df, gt=gt)

    return llik



In [ ]:
proposal = np.random.random(size=n_params)
print(get_log_prob(proposal))

In [ ]:
proposal = np.random.random(size=n_params)*2
print(get_log_prob(proposal))